# Подгружаем тестовый датасет

In [8]:
from pathlib import Path
import pandas as pd

DATASET_PATH = Path("../data/datasets/valid_pairs.csv")

dataset = pd.read_csv(DATASET_PATH, sep=",")

dataset

,text,aspect,label
0,"Отличный электив, если хочешь начать учить нем...",фильмы,0
1,"Данный электив я выбрал потому, что моя мать с...",материал__информация__темы,0
2,"Отличный электив, если хочешь начать учить нем...",эссе,0
3,"Интересный электив, очень многое узнаете об ис...",тесты,0
4,"Очень интересный электив, направленный на укре...",онлайн-курс,0
...,...,...,...
2370,"Из пюсов могу выделить то, что преподаватель в...",баллы__оценки,3
2371,Дмитрий Евгеньевич действительно очень хороший...,выступления,3
2372,"В целом, было несложно закрыть электив, если б...",материал__информация__темы,3
2373,"Скучный электив, лекции абсолютно не интересно...",тесты,3


In [9]:
aspects = list(dataset['aspect'].unique())
readable_aspects = [ s.replace("__", ", ").capitalize() for s in aspects ]
readable_aspect_to_aspect = {}
aspect_to_readable_aspect = {}
for a, b in zip(aspects, readable_aspects):
    readable_aspect_to_aspect[b] = a
    aspect_to_readable_aspect[a] = b

In [10]:
readable_aspect_to_aspect

{'Фильмы': 'фильмы',
 'Материал, информация, темы': 'материал__информация__темы',
 'Эссе': 'эссе',
 'Тесты': 'тесты',
 'Онлайн-курс': 'онлайн-курс',
 'Игры, интерактивность': 'игры__интерактивность',
 'Презентации': 'презентации',
 'Домашняя работа': 'домашняя работа',
 'Доклады': 'доклады',
 'Лекции': 'лекции',
 'Задания, задачи': 'задания__задачи',
 'Литература, учебники': 'литература__учебники',
 'Зачет, экзамен': 'зачет__экзамен',
 'Проекты': 'проекты',
 'Видео-уроки': 'видео-уроки',
 'Выступления': 'выступления',
 'Практики, семинары': 'практики__семинары',
 'Преподаватель': 'преподаватель',
 'Баллы, оценки': 'баллы__оценки'}

# Поиск аспектов на основе NLI

In [16]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import numpy as np

class MethodSimilarity():

    def transformers_tokenizer(self, sentence: str) -> torch.Tensor:
        
        tokens = self.transformers_auto_tokenizer(sentence, return_tensors='pt')
        vector = self.transformers_model(**tokens)[0].detach().squeeze()
        return torch.mean(vector, dim=0).numpy()
        # return self.model.encode(sentence, convert_to_tensor=True)

    def set_aspects(self, aspects_list=None):
        if aspects_list is not None:
            self.aspects_list = aspects_list
        else:
            self.aspects_list = readable_aspects

    def __init__(self, tokenizer="distiluse", min_similarity = 0.3):
        self.set_aspects(None)
        self.aspects_list = readable_aspects
        self.min_similarity = min_similarity
        if tokenizer == "distiluse":
            self.tokenizer = self.transformers_tokenizer
    
            self.transformers_auto_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")
            self.transformers_model = AutoModel.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")
            # self.model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
        elif tokenizer == "sbert-pq":
            self.tokenizer = self.transformers_tokenizer
            
            self.transformers_auto_tokenizer = AutoTokenizer.from_pretrained("inkoziev/sbert_pq")
            self.transformers_model = AutoModel.from_pretrained("inkoziev/sbert_pq")
            # self.model = SentenceTransformer("inkoziev/sbert_pq")
        elif tokenizer == "deberta":
            self.tokenizer = self.transformers_tokenizer
            
            self.transformers_auto_tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
            self.transformers_model = AutoModel.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
            # self.model = SentenceTransformer("MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
        else:
            self.tokenizer = None
            raise Exception("Invalid tokenizer.")
        
        self.cosine_similarity = self.calc_similarity
        # self.cosine_similarity = torch.nn.CosineSimilarity(dim=0)

    def calc_similarity(self, vector1, vector2):
        return np.dot(vector1, vector2) / \
               (np.linalg.norm(vector1) * np.linalg.norm(vector2))

    def process(self, text: str):
        aspects = []
        # Схожесть
        sentence_vector = self.tokenizer(text)
        similarities = [(aspect, self.cosine_similarity(self.tokenizer(aspect), sentence_vector))
                          for aspect in self.aspects_list]
        # ! DEBUG
        # print(text)
        # print(similarities)
        # ~ DEBUG
        for similarity in similarities:
            if similarity[1] > self.min_similarity:
                aspects.append(similarity[0])
        return aspects

In [17]:
search = MethodSimilarity(tokenizer="distiluse", min_similarity=0.2)

In [18]:
search.process("Лекции были скучными")

['Фильмы',
 'Материал, информация, темы',
 'Эссе',
 'Тесты',
 'Онлайн-курс',
 'Игры, интерактивность',
 'Презентации',
 'Домашняя работа',
 'Доклады',
 'Лекции',
 'Задания, задачи',
 'Литература, учебники',
 'Зачет, экзамен',
 'Проекты',
 'Видео-уроки',
 'Выступления',
 'Практики, семинары',
 'Преподаватель',
 'Баллы, оценки']

In [19]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka')
model = AutoModelForSequenceClassification.from_pretrained('seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka', return_dict=True)

seninoseno_labels_to_our = {
    0: 1,
    1: 2,
    2: 3,
}

def predict_sentiment(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return seninoseno_labels_to_our[predicted[0]]

In [20]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch

# model_name = "seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name)

# # set device to GPU
# device = torch.device('cuda')

# # put model on GPU (is done in-place)
# model.to(device)

# seninoseno_labels_to_our = {
#     'NEUTRAL': 1,
#     'POSITIVE': 2,
#     'NEGATIVE': 3,
# }

# def predict_sentiment(text):
#     encoding = tokenizer(text, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
#     # put data on GPU
#     for k,v in encoding.items():
#          encoding[k] = v.to(device)
#     # forward pass
#     outputs = model(**encoding)
#     # get predicted class indices
#     predicted_class_indices = outputs.logits.argmax(-1).tolist()
#     # turn into actual class names
#     predicted_classes = [model.config.id2label[label] for label in predicted_class_indices]
#     return predicted_classes[0]

In [21]:
predict_sentiment("Практики были отличными")

2

In [22]:
import razdel

def razdel_texta(text: str) -> list[str]:
    return [_.text for _ in razdel.sentenize(text)]

In [23]:
razdel_texta("Это первое предложение. А это второе.")

['Это первое предложение.', 'А это второе.']

In [24]:
def predict(text, aspect) -> int:
    sentences = razdel_texta(text)
    for sentence in sentences:
        found_aspects = search.process(sentence)
        if aspect_to_readable_aspect[aspect] in found_aspects:
            sentiment = predict_sentiment(sentence)
            return sentiment
    return 0

In [27]:
# предиктим аспекты

import pickle
from tqdm import tqdm

#1750:
#df_slice = dataset[1650:].sample(frac=1.0)
df_slice = dataset.sample(frac=1.0)

y_pred = []
for s, a in tqdm(zip(df_slice["text"], df_slice["aspect"]), total=len(df_slice)):
    y_pred.append(predict(s, a))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2375/2375 [45:22<00:00,  1.15s/it]


In [38]:
OUTPUT_PREDICTED_DIR = Path("../output/test-old-method-absa-accuracy")
OUTPUT_FILENAME = "predicted.bin"
OUTPUT_PREDICTED_PATH = OUTPUT_PREDICTED_DIR / OUTPUT_FILENAME
OUTPUT_PREDICTED_DIR.mkdir(parents=True, exist_ok=True)

In [39]:
# и сохраняем в файл на будещее.
with open(OUTPUT_PREDICTED_PATH, "wb") as f:
    pickle.dump(y_pred, f)

In [44]:
# подгружаем запредикченые значения
with open(OUTPUT_PREDICTED_PATH, "rb") as f:
    y_pred = pickle.load(f)
print(len(y_pred))

2375


In [45]:
y_true = list(df_slice["label"])
print(len(y_true))

2375


In [47]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("F1 ", f1_score(y_true=y_true, y_pred=y_pred, average="macro"))
print("Precision ", precision_score(y_true=y_true, y_pred=y_pred, average="macro"))
print("Recall ", recall_score(y_true=y_true, y_pred=y_pred, average="macro"))

F1  0.41829607270292685
Precision  0.3978856736731679
Recall  0.4579162819967989
